In [ ]:
!pip install tensorflow pandas numpy matplotlib scikit-learn


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt


In [ ]:
# Load CSV
csv_path = "/content/image_data.csv"   # CHANGE THIS
df = pd.read_csv(csv_path)

print(df.head())


In [ ]:
def preprocess_pixels(pixel_string):
    pixels = np.array(pixel_string.split(), dtype='float32')
    pixels = pixels.reshape(48, 48, 1) / 255.0
    return pixels


In [ ]:
expected_pixels_count = 48 * 48
valid_pixels_mask = df['pixels'].apply(lambda x: len(x.split())) == expected_pixels_count
df_filtered = df[valid_pixels_mask]

X = np.array([preprocess_pixels(p) for p in df_filtered['pixels']])
y = df_filtered['emotion'].values

In [ ]:
# Re-initialize y from the original filtered DataFrame to ensure it's a 1D array of labels.
y = df_filtered['emotion'].values

encoder = OneHotEncoder(sparse_output=False)
y = encoder.fit_transform(y.reshape(-1, 1))

print("Images shape:", X.shape)
print("Labels shape:", y.shape)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(48,48,1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(y.shape[1], activation='softmax')
])


In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)


In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=15,          # ↓ from 30
    batch_size=64,
    validation_data=(X_val, y_val),
    callbacks=[early_stop]
)


In [ ]:
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.legend()
plt.title("Accuracy")

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title("Loss")

plt.show()


In [ ]:
sample = X_val[0:1]
pred = model.predict(sample)

print("Emotion probabilities:", pred)


In [ ]:
def emotion_dispersion(probabilities):
    mean = np.mean(probabilities)
    dispersion = np.sqrt(np.mean((probabilities - mean)**2))
    return dispersion


In [ ]:
disp = emotion_dispersion(pred[0])
print("Emotion Dispersion:", disp)


In [ ]:
model.save("emotion_model_6017.h5")